In [ ]:
# %% [markdown]
# # Home Credit Default Risk - Feature Engineering
# 
# ## 🛠️ Overview
# This notebook creates new features by:
# 1. Aggregating data from multiple sources
# 2. Creating interaction features
# 3. Handling categorical variables
# 4. Creating domain-specific features
# 
# ## 📊 Data Sources Used
# 1. application_train/test.csv (main data)
# 2. bureau.csv (external loans)
# 3. previous_application.csv (previous Home Credit loans)
# 4. Other tables for payment behavior

# %% [markdown]
# ## 📦 Setup and Imports

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import project modules
import sys
sys.path.append('../src')
from utils import load_config, load_data, reduce_memory_usage
from feature_engineering import FeatureEngineer

# Set display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# %%
# Load configuration
config = load_config()
print("Configuration loaded successfully")

# %%
# Initialize feature engineer
engineer = FeatureEngineer()

# %% [markdown]
# ## 📁 Load Data

# %%
# Load main application data
print("Loading application data...")
app_train = load_data(config['files']['application_train'])
app_test = load_data(config['files']['application_test'])

print(f"Training data shape: {app_train.shape}")
print(f"Test data shape: {app_test.shape}")

# %%
# Load bureau data
print("\nLoading bureau data...")
bureau = load_data(config['files']['bureau'])

print(f"Bureau data shape: {bureau.shape}")
print(f"Unique clients in bureau: {bureau['SK_ID_CURR'].nunique()}")
print(f"Unique loans in bureau: {bureau['SK_ID_BUREAU'].nunique()}")

# %%
# Load previous application data
print("\nLoading previous application data...")
prev_app = load_data(config['files']['previous_application'])

print(f"Previous application data shape: {prev_app.shape}")
print(f"Unique clients in previous apps: {prev_app['SK_ID_CURR'].nunique()}")
print(f"Unique previous applications: {prev_app['SK_ID_PREV'].nunique()}")

# %% [markdown]
# ## 🎯 Create Application Features

# %%
def create_application_features(df):
    """Create features from main application data."""
    features_df = df.copy()
    
    print(f"Original columns: {len(features_df.columns)}")
    
    # 1. Income-related features
    if 'AMT_INCOME_TOTAL' in df.columns:
        # Income to credit ratio
        if 'AMT_CREDIT' in df.columns:
            features_df['INCOME_CREDIT_RATIO'] = df['AMT_INCOME_TOTAL'] / (df['AMT_CREDIT'] + 1)
            features_df['INCOME_CREDIT_RATIO'].replace([np.inf, -np.inf], np.nan, inplace=True)
        
        # Income per person
        if 'CNT_FAM_MEMBERS' in df.columns:
            features_df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / (df['CNT_FAM_MEMBERS'] + 1)
        
        # Income to annuity ratio
        if 'AMT_ANNUITY' in df.columns:
            features_df['INCOME_ANNUITY_RATIO'] = df['AMT_INCOME_TOTAL'] / (df['AMT_ANNUITY'] + 1)
            features_df['INCOME_ANNUITY_RATIO'].replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # 2. Credit-related features
    if 'AMT_CREDIT' in df.columns:
        # Credit to goods price ratio
        if 'AMT_GOODS_PRICE' in df.columns:
            features_df['CREDIT_GOODS_RATIO'] = df['AMT_CREDIT'] / (df['AMT_GOODS_PRICE'] + 1)
        
        # Credit to income ratio (already have inverse)
        features_df['CREDIT_INCOME_RATIO'] = df['AMT_CREDIT'] / (df['AMT_INCOME_TOTAL'] + 1)
    
    # 3. Age-related features
    if 'DAYS_BIRTH' in df.columns:
        features_df['AGE'] = -df['DAYS_BIRTH'] / 365.25
        features_df['AGE'] = features_df['AGE'].round(0)
        
        # Age groups
        features_df['AGE_GROUP'] = pd.cut(
            features_df['AGE'],
            bins=[0, 25, 35, 45, 55, 65, 100],
            labels=['18-25', '26-35', '36-45', '46-55', '56-65', '65+']
        )
    
    # 4. Employment features
    if 'DAYS_EMPLOYED' in df.columns:
        # Flag for unrealistic employment days
        features_df['DAYS_EMPLOYED_ANOMALY'] = (df['DAYS_EMPLOYED'] == 365243).astype(int)
        
        # Employment length in years
        features_df['YEARS_EMPLOYED'] = -df['DAYS_EMPLOYED'].replace(365243, np.nan) / 365.25
        
        # Employment status
        features_df['EMPLOYED'] = (features_df['YEARS_EMPLOYED'] > 0).astype(int)
    
    # 5. Document features
    doc_cols = [col for col in df.columns if 'FLAG_DOCUMENT' in col]
    if doc_cols:
        features_df['DOCUMENT_COUNT'] = df[doc_cols].sum(axis=1)
        features_df['HAS_DOCUMENTS'] = (features_df['DOCUMENT_COUNT'] > 0).astype(int)
    
    # 6. Contact features
    contact_cols = ['FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 
                   'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL']
    contact_cols = [col for col in contact_cols if col in df.columns]
    if contact_cols:
        features_df['CONTACT_INFO_COUNT'] = df[contact_cols].sum(axis=1)
        features_df['HAS_CONTACT_INFO'] = (features_df['CONTACT_INFO_COUNT'] > 0).astype(int)
    
    # 7. Car and property ownership
    if 'FLAG_OWN_CAR' in df.columns:
        features_df['OWNS_CAR'] = (df['FLAG_OWN_CAR'] == 'Y').astype(int)
    
    if 'FLAG_OWN_REALTY' in df.columns:
        features_df['OWNS_PROPERTY'] = (df['FLAG_OWN_REALTY'] == 'Y').astype(int)
    
    # 8. Family features
    if 'CNT_CHILDREN' in df.columns:
        features_df['HAS_CHILDREN'] = (df['CNT_CHILDREN'] > 0).astype(int)
    
    if 'CNT_FAM_MEMBERS' in df.columns and 'CNT_CHILDREN' in df.columns:
        features_df['ADULTS_IN_FAMILY'] = df['CNT_FAM_MEMBERS'] - df['CNT_CHILDREN']
    
    print(f"New columns: {len(features_df.columns)}")
    print(f"Features created: {len(features_df.columns) - len(df.columns)}")
    
    return features_df

# %%
# Create application features
print("Creating application features for training data...")
app_train_features = create_application_features(app_train)

print("\nCreating application features for test data...")
app_test_features = create_application_features(app_test)

# %%
# Show new features
new_features = [col for col in app_train_features.columns if col not in app_train.columns]
print(f"\nNew application features created: {len(new_features)}")
print("Top 10 new features:")
for feature in new_features[:10]:
    print(f"  - {feature}")

# %% [markdown]
# ## 🏦 Create Bureau Features

# %%
def create_bureau_features(application_df, bureau_df):
    """Create features from bureau data."""
    
    print(f"\nProcessing {len(bureau_df):,} bureau records...")
    
    # Create a copy to avoid modifying original
    bureau = bureau_df.copy()
    
    # Initialize aggregations dictionary
    bureau_agg = {}
    
    # 1. Basic counts
    bureau_agg['BUREAU_LOAN_COUNT'] = bureau.groupby('SK_ID_CURR')['SK_ID_BUREAU'].count()
    bureau_agg['BUREAU_ACTIVE_COUNT'] = bureau[bureau['CREDIT_ACTIVE'] == 'Active'].groupby('SK_ID_CURR').size()
    bureau_agg['BUREAU_CLOSED_COUNT'] = bureau[bureau['CREDIT_ACTIVE'] == 'Closed'].groupby('SK_ID_CURR').size()
    
    # 2. Credit amount aggregations
    if 'AMT_CREDIT_SUM' in bureau.columns:
        bureau_agg.update({
            'BUREAU_CREDIT_SUM_MEAN': bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].mean(),
            'BUREAU_CREDIT_SUM_MAX': bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].max(),
            'BUREAU_CREDIT_SUM_MIN': bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].min(),
            'BUREAU_CREDIT_SUM_SUM': bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].sum(),
        })
    
    # 3. Debt aggregations
    if 'AMT_CREDIT_SUM_DEBT' in bureau.columns:
        bureau_agg.update({
            'BUREAU_DEBT_MEAN': bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].mean(),
            'BUREAU_DEBT_MAX': bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].max(),
            'BUREAU_DEBT_SUM': bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].sum(),
        })
        
        # Debt to credit ratio
        if 'AMT_CREDIT_SUM' in bureau.columns:
            bureau['DEBT_CREDIT_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / (bureau['AMT_CREDIT_SUM'] + 1)
            bureau_agg['BUREAU_DEBT_CREDIT_RATIO_MEAN'] = bureau.groupby('SK_ID_CURR')['DEBT_CREDIT_RATIO'].mean()
    
    # 4. Overdue days
    if 'CREDIT_DAY_OVERDUE' in bureau.columns:
        bureau_agg.update({
            'BUREAU_MAX_OVERDUE_DAYS': bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].max(),
            'BUREAU_MEAN_OVERDUE_DAYS': bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].mean(),
        })
        
        # Flag for any overdue
        bureau['HAS_OVERDUE'] = (bureau['CREDIT_DAY_OVERDUE'] > 0).astype(int)
        bureau_agg['BUREAU_HAS_OVERDUE_COUNT'] = bureau.groupby('SK_ID_CURR')['HAS_OVERDUE'].sum()
    
    # 5. Credit duration
    if 'DAYS_CREDIT' in bureau.columns:
        bureau_agg.update({
            'BUREAU_CREDIT_DAYS_MEAN': bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].mean(),
            'BUREAU_CREDIT_DAYS_MAX': bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].max(),
            'BUREAU_CREDIT_DAYS_MIN': bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].min(),
        })
    
    # 6. Credit type diversity
    if 'CREDIT_TYPE' in bureau.columns:
        bureau_agg['BUREAU_CREDIT_TYPES_COUNT'] = bureau.groupby('SK_ID_CURR')['CREDIT_TYPE'].nunique()
        
        # Most common credit type
        most_common = bureau.groupby(['SK_ID_CURR', 'CREDIT_TYPE']).size().reset_index(name='count')
        most_common = most_common.loc[most_common.groupby('SK_ID_CURR')['count'].idxmax()]
        most_common = most_common.set_index('SK_ID_CURR')['CREDIT_TYPE']
        bureau_agg['BUREAU_MAIN_CREDIT_TYPE'] = most_common
    
    # 7. Currency diversity
    if 'CREDIT_CURRENCY' in bureau.columns:
        bureau_agg['BUREAU_CURRENCY_COUNT'] = bureau.groupby('SK_ID_CURR')['CREDIT_CURRENCY'].nunique()
    
    # Create aggregated DataFrame
    bureau_features = pd.DataFrame(bureau_agg).reset_index()
    
    print(f"Created {len(bureau_features.columns) - 1} bureau features")
    
    # Merge with application data
    result_df = application_df.merge(bureau_features, on='SK_ID_CURR', how='left')
    
    # Fill missing values for applicants with no bureau data
    bureau_cols = [col for col in bureau_features.columns if col != 'SK_ID_CURR']
    for col in bureau_cols:
        if col in result_df.columns:
            if result_df[col].dtype in ['float64', 'float32', 'float16']:
                result_df[col] = result_df[col].fillna(0)
            elif result_df[col].dtype == 'object':
                result_df[col] = result_df[col].fillna('No Bureau Data')
    
    return result_df

# %%
# Create bureau features
print("Creating bureau features for training data...")
train_with_bureau = create_bureau_features(app_train_features, bureau)

print("\nCreating bureau features for test data...")
test_with_bureau = create_bureau_features(app_test_features, bureau)

# %%
# Show new bureau features
bureau_features = [col for col in train_with_bureau.columns if col not in app_train_features.columns]
print(f"\nNew bureau features created: {len(bureau_features)}")
print("Sample bureau features:")
for feature in bureau_features[:10]:
    print(f"  - {feature}")

# %% [markdown]
# ## 🏠 Create Previous Application Features

# %%
def create_previous_application_features(application_df, prev_app_df):
    """Create features from previous Home Credit applications."""
    
    print(f"\nProcessing {len(prev_app_df):,} previous applications...")
    
    prev = prev_app_df.copy()
    
    # Initialize aggregations
    prev_agg = {}
    
    # 1. Basic counts
    prev_agg['PREV_APPLICATION_COUNT'] = prev.groupby('SK_ID_CURR')['SK_ID_PREV'].count()
    
    # 2. Application status features
    if 'NAME_CONTRACT_STATUS' in prev.columns:
        # Count by status
        for status in prev['NAME_CONTRACT_STATUS'].unique():
            status_count = prev[prev['NAME_CONTRACT_STATUS'] == status].groupby('SK_ID_CURR').size()
            prev_agg[f'PREV_STATUS_{status.upper()}_COUNT'] = status_count
        
        # Approved applications
        approved = prev[prev['NAME_CONTRACT_STATUS'] == 'Approved'].groupby('SK_ID_CURR').size()
        prev_agg['PREV_APPROVED_COUNT'] = approved
        
        # Approval rate
        prev_agg['PREV_APPROVAL_RATE'] = approved / prev_agg['PREV_APPLICATION_COUNT']
    
    # 3. Credit amount aggregations
    if 'AMT_CREDIT' in prev.columns:
        prev_agg.update({
            'PREV_AMT_CREDIT_MEAN': prev.groupby('SK_ID_CURR')['AMT_CREDIT'].mean(),
            'PREV_AMT_CREDIT_MAX': prev.groupby('SK_ID_CURR')['AMT_CREDIT'].max(),
            'PREV_AMT_CREDIT_MIN': prev.groupby('SK_ID_CURR')['AMT_CREDIT'].min(),
            'PREV_AMT_CREDIT_SUM': prev.groupby('SK_ID_CURR')['AMT_CREDIT'].sum(),
        })
    
    # 4. Application amount aggregations
    if 'AMT_APPLICATION' in prev.columns:
        prev_agg.update({
            'PREV_AMT_APPLICATION_MEAN': prev.groupby('SK_ID_CURR')['AMT_APPLICATION'].mean(),
            'PREV_AMT_APPLICATION_MAX': prev.groupby('SK_ID_CURR')['AMT_APPLICATION'].max(),
            'PREV_AMT_APPLICATION_SUM': prev.groupby('SK_ID_CURR')['AMT_APPLICATION'].sum(),
        })
        
        # Application to credit ratio
        if 'AMT_CREDIT' in prev.columns:
            prev['APPLICATION_CREDIT_RATIO'] = prev['AMT_APPLICATION'] / (prev['AMT_CREDIT'] + 1)
            prev_agg['PREV_APP_CREDIT_RATIO_MEAN'] = prev.groupby('SK_ID_CURR')['APPLICATION_CREDIT_RATIO'].mean()
    
    # 5. Annuity aggregations
    if 'AMT_ANNUITY' in prev.columns:
        prev_agg.update({
            'PREV_AMT_ANNUITY_MEAN': prev.groupby('SK_ID_CURR')['AMT_ANNUITY'].mean(),
            'PREV_AMT_ANNUITY_MAX': prev.groupby('SK_ID_CURR')['AMT_ANNUITY'].max(),
            'PREV_AMT_ANNUITY_SUM': prev.groupby('SK_ID_CURR')['AMT_ANNUITY'].sum(),
        })
    
    # 6. Days decision features
    if 'DAYS_DECISION' in prev.columns:
        prev_agg.update({
            'PREV_DAYS_DECISION_MEAN': prev.groupby('SK_ID_CURR')['DAYS_DECISION'].mean(),
            'PREV_DAYS_DECISION_MAX': prev.groupby('SK_ID_CURR')['DAYS_DECISION'].max(),
            'PREV_DAYS_DECISION_MIN': prev.groupby('SK_ID_CURR')['DAYS_DECISION'].min(),
        })
    
    # 7. Contract type features
    if 'NAME_CONTRACT_TYPE' in prev.columns:
        # Count of contract types
        prev_agg['PREV_CONTRACT_TYPES_COUNT'] = prev.groupby('SK_ID_CURR')['NAME_CONTRACT_TYPE'].nunique()
        
        # Most common contract type
        contract_counts = prev.groupby(['SK_ID_CURR', 'NAME_CONTRACT_TYPE']).size().reset_index(name='count')
        most_common_contract = contract_counts.loc[contract_counts.groupby('SK_ID_CURR')['count'].idxmax()]
        most_common_contract = most_common_contract.set_index('SK_ID_CURR')['NAME_CONTRACT_TYPE']
        prev_agg['PREV_MAIN_CONTRACT_TYPE'] = most_common_contract
    
    # Create aggregated DataFrame
    prev_features = pd.DataFrame(prev_agg).reset_index()
    
    print(f"Created {len(prev_features.columns) - 1} previous application features")
    
    # Merge with application data
    result_df = application_df.merge(prev_features, on='SK_ID_CURR', how='left')
    
    # Fill missing values
    prev_cols = [col for col in prev_features.columns if col != 'SK_ID_CURR']
    for col in prev_cols:
        if col in result_df.columns:
            if result_df[col].dtype in ['float64', 'float32', 'float16']:
                result_df[col] = result_df[col].fillna(0)
            elif result_df[col].dtype == 'object':
                result_df[col] = result_df[col].fillna('No Previous Apps')
    
    return result_df

# %%
# Create previous application features
print("Creating previous application features for training data...")
train_with_prev = create_previous_application_features(train_with_bureau, prev_app)

print("\nCreating previous application features for test data...")
test_with_prev = create_previous_application_features(test_with_bureau, prev_app)

# %%
# Show new previous application features
prev_features = [col for col in train_with_prev.columns if col not in train_with_bureau.columns]
print(f"\nNew previous application features created: {len(prev_features)}")
print("Sample previous application features:")
for feature in prev_features[:10]:
    print(f"  - {feature}")

# %% [markdown]
# ## 🔧 Feature Selection and Cleaning

# %%
def clean_and_select_features(df, is_train=True):
    """Clean features and select important ones."""
    
    print(f"\nCleaning and selecting features...")
    print(f"Original shape: {df.shape}")
    
    features_df = df.copy()
    
    # 1. Drop columns with too many missing values (>80%)
    missing_threshold = 0.8
    missing_ratios = features_df.isnull().sum() / len(features_df)
    columns_to_drop = missing_ratios[missing_ratios > missing_threshold].index.tolist()
    
    if columns_to_drop:
        features_df = features_df.drop(columns=columns_to_drop)
        print(f"  Dropped {len(columns_to_drop)} columns with >{missing_threshold*100:.0f}% missing values")
    
    # 2. Drop constant columns
    constant_columns = []
    for col in features_df.columns:
        if col not in ['SK_ID_CURR', 'TARGET']:
            if features_df[col].nunique() <= 1:
                constant_columns.append(col)
    
    if constant_columns:
        features_df = features_df.drop(columns=constant_columns)
        print(f"  Dropped {len(constant_columns)} constant columns")
    
    # 3. Handle missing values
    for col in features_df.columns:
        if col not in ['SK_ID_CURR', 'TARGET']:
            if features_df[col].isnull().any():
                if features_df[col].dtype in ['float64', 'float32', 'float16', 'int64', 'int32', 'int16', 'int8']:
                    # Fill numeric columns with median
                    features_df[col] = features_df[col].fillna(features_df[col].median())
                elif features_df[col].dtype == 'object':
                    # Fill categorical columns with mode
                    features_df[col] = features_df[col].fillna(features_df[col].mode()[0])
    
    print(f"  Filled missing values")
    
    # 4. Encode categorical variables
    categorical_cols = features_df.select_dtypes(include=['object']).columns.tolist()
    
    if categorical_cols:
        print(f"  Encoding {len(categorical_cols)} categorical columns")
        
        # Label encoding for tree-based models
        for col in categorical_cols:
            if col not in ['SK_ID_CURR', 'TARGET']:
                features_df[col] = features_df[col].astype('category').cat.codes
    
    # 5. Remove highly correlated features
    if is_train and 'TARGET' in features_df.columns:
        # Calculate correlation matrix
        numeric_cols = features_df.select_dtypes(include=[np.number]).columns.tolist()
        numeric_cols = [col for col in numeric_cols if col not in ['SK_ID_CURR', 'TARGET']]
        
        if numeric_cols:
            corr_matrix = features_df[numeric_cols].corr().abs()
            
            # Select upper triangle of correlation matrix
            upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
            
            # Find features with correlation greater than 0.9
            to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
            
            if to_drop:
                features_df = features_df.drop(columns=to_drop)
                print(f"  Dropped {len(to_drop)} highly correlated features")
    
    print(f"Final shape: {features_df.shape}")
    
    return features_df

# %%
# Clean and select features for training data
print("Processing training data...")
train_final = clean_and_select_features(train_with_prev, is_train=True)

print("\nProcessing test data...")
test_final = clean_and_select_features(test_with_prev, is_train=False)

# Ensure test data has same columns as train data (except TARGET)
train_cols = [col for col in train_final.columns if col != 'TARGET']
test_final = test_final[train_cols + ['SK_ID_CURR']]

# %% [markdown]
# ## 📊 Feature Analysis

# %%
# Analyze feature importance with simple correlation
if 'TARGET' in train_final.columns:
    print("Analyzing feature correlations with target...")
    
    # Calculate correlation with target
    numeric_cols = train_final.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [col for col in numeric_cols if col not in ['SK_ID_CURR', 'TARGET']]
    
    correlations = []
    for col in numeric_cols:
        corr = train_final[col].corr(train_final['TARGET'])
        correlations.append((col, abs(corr)))
    
    # Sort by absolute correlation
    correlations.sort(key=lambda x: x[1], reverse=True)
    
    print(f"\nTop 10 features most correlated with TARGET:")
    for col, corr in correlations[:10]:
        print(f"  {col}: {corr:.4f}")
    
    print(f"\nBottom 10 features least correlated with TARGET:")
    for col, corr in correlations[-10:]:
        print(f"  {col}: {corr:.4f}")
    
    # Plot top correlated features
    top_features = [col for col, _ in correlations[:20]]
    
    if top_features:
        fig, axes = plt.subplots(5, 4, figsize=(20, 15))
        axes = axes.flatten()
        
        for i, feature in enumerate(top_features[:20]):
            if i < len(axes):
                # Plot distribution by target
                for target_val in [0, 1]:
                    subset = train_final[train_final['TARGET'] == target_val][feature].dropna()
                    axes[i].hist(subset, alpha=0.5, label=f'Target={target_val}', bins=30)
                
                axes[i].set_title(f'{feature}\nCorr: {correlations[i][1]:.3f}')
                axes[i].set_xlabel('Value')
                axes[i].set_ylabel('Frequency')
                axes[i].legend()
                axes[i].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# %% [markdown]
# ## 💾 Save Engineered Features

# %%
# Save features to disk
print("\nSaving engineered features...")

# Create features directory
import os
os.makedirs('../features', exist_ok=True)

# Save training features
train_final.to_pickle('../features/train_features.pkl')
print(f"Training features saved: {train_final.shape}")

# Save test features
test_final.to_pickle('../features/test_features.pkl')
print(f"Test features saved: {test_final.shape}")

# Save feature list
feature_list = [col for col in train_final.columns if col not in ['SK_ID_CURR', 'TARGET']]
with open('../features/feature_list.txt', 'w') as f:
    for feature in feature_list:
        f.write(f"{feature}\n")

print(f"Feature list saved: {len(feature_list)} features")

# %% [markdown]
# ## 📝 Summary

# %%
print("=" * 60)
print("FEATURE ENGINEERING SUMMARY")
print("=" * 60)

print(f"\n1. APPLICATION FEATURES:")
print(f"   - Original: {len(app_train.columns)} columns")
print(f"   - New: {len(app_train_features.columns)} columns")
print(f"   - Created: {len(app_train_features.columns) - len(app_train.columns)} new features")

print(f"\n2. BUREAU FEATURES:")
print(f"   - Added: {len(bureau_features)} bureau features")

print(f"\n3. PREVIOUS APPLICATION FEATURES:")
print(f"   - Added: {len(prev_features)} previous application features")

print(f"\n4. FINAL DATASETS:")
print(f"   - Training: {train_final.shape[0]} samples, {train_final.shape[1]} features")
print(f"   - Test: {test_final.shape[0]} samples, {test_final.shape[1]} features")

print(f"\n5. NEXT STEPS:")
print(f"   - Model training with {len(feature_list)} features")
print(f"   - Feature importance analysis")
print(f"   - Hyperparameter tuning")

# Save summary
summary = {
    'train_samples': train_final.shape[0],
    'train_features': train_final.shape[1],
    'test_samples': test_final.shape[0],
    'test_features': test_final.shape[1],
    'feature_list_size': len(feature_list)
}

import json
with open('../features/engineering_summary.json', 'w') as f:
    json.dump(summary, f, indent=4)

print("\nSummary saved to features/engineering_summary.json")